In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql
import re
from dotenv import load_dotenv
load_dotenv()
import os

In [2]:
db_connection_str = os.getenv("DB_CONNECTION_STR")

db_connection = create_engine(db_connection_str)

df = pd.read_sql('SELECT * FROM crash_db_Los_Angles', con=db_connection)
df = df.replace('',np.nan)

In [3]:
df.shape

(510209, 22)

In [4]:
df.head()

,cid,createDate,createBy,dataFrom,DR_Number,Date_Reported,Date_Occurred,Time_Occurred,Area_ID,Area_Name,...,Crime_Code_Description,MO_Codes,Victim_Age,Victim_Sex,Victim_Descent,Premise_Code,Premise_Description,Address,Cross_Street,Location
0,1000000001,2020-01-28 05:23:19,David,Los_Angles,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,...,Crime Code Description,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Code,Premise Description,Address,Cross Street,Location
1,1000000002,2020-01-28 05:23:19,David,Los_Angles,190211527,05/18/2019,05/18/2019,0110,02,Rampart,...,TRAFFIC COLLISION,3101 3401 3701 3004 3030,33,F,H,101,STREET,UNION,JAMES M WOOD,"(34.0508, -118.2731)"
2,1000000003,2020-01-28 05:23:19,David,Los_Angles,110518459,10/11/2011,10/11/2011,1735,05,Harbor,...,TRAFFIC COLLISION,0506,27,F,O,101,STREET,VERMONT,KNOX,"(33.8542, -118.2905)"
3,1000000004,2020-01-28 05:23:19,David,Los_Angles,180618348,08/09/2018,08/09/2018,1830,06,Hollywood,...,TRAFFIC COLLISION,3004 3028 3030 3035 3036 3101 3401 3701 4006,33,M,W,101,STREET,FRANKLIN AV,ARGYLE,"(34.1052, -118.3252)"
4,1000000005,2020-01-28 05:23:19,David,Los_Angles,120312271,05/09/2012,05/09/2012,0845,03,Southwest,...,TRAFFIC COLLISION,NaN,51,M,B,101,STREET,LA BREA AV,JEFFERSON BL,"(34.0255, -118.3548)"


In [5]:
df = df[1:]

In [6]:
df.columns

Index(['cid', 'createDate', 'createBy', 'dataFrom', 'DR_Number',
       'Date_Reported', 'Date_Occurred', 'Time_Occurred', 'Area_ID',
       'Area_Name', 'Reporting_District', 'Crime_Code',
       'Crime_Code_Description', 'MO_Codes', 'Victim_Age', 'Victim_Sex',
       'Victim_Descent', 'Premise_Code', 'Premise_Description', 'Address',
       'Cross_Street', 'Location'],
      dtype='object')

In [7]:
df2 = pd.DataFrame()
df2['cid'] = df.cid
df2['city'] = df.dataFrom
df2['crash_time'] = df.Date_Occurred + ' ' + df.Time_Occurred
df2['street_name'] = df['Address']
df2['latitude'] = df['Location'].str.split(r"[(,)]").str[1]
df2['longitude'] = df['Location'].str.split(r"[(,)]").str[2]
df2['injured_count'] = 0

df2['crash_time'] = pd.to_datetime(df2['crash_time'])
df2['crash_date'] = pd.to_datetime(df2['crash_time'].dt.date)

In [8]:
df2.head()

,cid,city,crash_time,street_name,latitude,longitude,injured_count,crash_date
1,1000000002,Los_Angles,2019-05-18 01:10:00,UNION,34.0508,-118.2731,0,2019-05-18
2,1000000003,Los_Angles,2011-10-11 17:35:00,VERMONT,33.8542,-118.2905,0,2011-10-11
3,1000000004,Los_Angles,2018-08-09 18:30:00,FRANKLIN AV,34.1052,-118.3252,0,2018-08-09
4,1000000005,Los_Angles,2012-05-09 08:45:00,LA BREA AV,34.0255,-118.3548,0,2012-05-09
5,1000000006,Los_Angles,2012-06-13 07:45:00,48TH ST,33.9997,-118.2827,0,2012-06-13


In [9]:
df2.to_sql('cleaned_crash_Los_Angeles',con = db_connection, if_exists="replace",index=False)

In [ ]:
df2.to_sql('home_allcleanedcrashdata',con = db_connection, if_exists="append",index=False)